Part 3 - Graph Analytics with Cypher and Neo4j
==============================================

<center><img src="images/Neo4j-logo_color.png" width="600px" /></center>

In this section we will use Neo4j and Bloom to learn Graph Analytics against the Open Sanctions database as outlined on Github at [opensanctions/offshore-graph](https://github.com/opensanctions/offshore-graph#sanctionsoffshores-graph-demo).

The following resources will assist you in learning Cypher to query Neo4j and other graph databases via [openCypher](https://opencypher.org/) ([github](https://github.com/opencypher)):

* [Neo4j Cypher Manual - Overview](https://neo4j.com/docs/cypher-manual/current/introduction/cypher_overview/)
* [Neo4j Cypher Manual - Cypher and Neo4j](https://neo4j.com/docs/cypher-manual/current/introduction/cypher_neo4j/)
* [Neo4j 5 Cypher Cheat Sheet](https://neo4j.com/docs/cypher-cheat-sheet/5/auradb-enterprise/)
* [Neo4j Graph Academy Cypher Fundamentals - 1 Hour Video](https://graphacademy.neo4j.com/courses/cypher-fundamentals/)
* [Neo4j Cypher Manual PDF](https://neo4j.com/docs/pdf/neo4j-cypher-manual-5.pdf)
* [Neo4j Getting Started - Query a Neo4j database using Cypher](https://neo4j.com/docs/getting-started/cypher-intro/)
* [Bite-Sized Neo4j for Data Scientists](https://neo4j.com/video/bite-sized-neo4j-for-data-scientists/) [[github code](https://github.com/cj2001/bite_sized_data_science)]
* [Using Neo4j from Python](https://neo4j.com/developer/python/)

We are going to call Neo4j from Python in this notebook. For simplicity's sake in setting up the course's software, I build a docker image and we run Neo4j Community on Docker via our [docker-compose.yml](docker-compose.yml) file. Docker only supports Neo4j Community unless you get a license (checkout [Neo4j for Startups](https://neo4j.com/startups/)) If you install [Neo4j Desktop](https://neo4j.com/download/) - which comes with a [developers' license]() to [Neo4j Enterprise]() - you can use [Neo4j Bloom](https://neo4j.com/product/bloom/). It is snazzy! :) It can visualize schemas and query results.

# Connecting to Neo4j from Python

We are going to be using two ways to query Cypher.

1) The [neo4j]() PyPi module [[github](https://github.com/neo4j/neo4j-python-driver)], [[Neo4j docs](https://neo4j.com/docs/api/python-driver/current/)]
2) Neo4j's Graphistry Integration [[example notebook](https://github.com/graphistry/pygraphistry/blob/master/demos/demos_databases_apis/neo4j/official/graphistry_bolt_tutorial_public.ipynb)]

We're going to use one or the other as the instructor prefers, depending on how important it is to visualize our results in a table, chart or as a network visualizaton using [Graphistry](https://graphistry.com).

## Describe Our Network

When you start working with a new graph database, you need to get oriented by describing the schema, then the network, then sampling some data and finally with some exploratory data analysis. From there you know enough to begin your own workflows.

### Describing our Property Graph Model

Let's start by describing our network. What kinds of nodes are there?

In [ ]:
from neo4j import GraphDatabase, RoutingControl

URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "password")

MATCH (n) RETURN distinct labels(n), count(*)